# Trabalho Prático 3
## ISEL - LEIM

## Docente - Gonçalo Marques

### Trabalho realizado por:
Mª Luísa Melo e Sampaio Nº50799
André Nº
Carlos Ceita Nº50950

In [ ]:
#imports
from time import time
from os import path
import numpy as np
import cv2
import matplotlib.pyplot as plt

In [ ]:
# ler um dos ficheiros
x = np.fromfile("LenaGray.tif", dtype=uint8)

# calcular histograma
h, bins, patches = plt.hist(x, 256, [0, 256])

# gerar código huffman
to = time()
tabela_codigo = gen_huff_table(np.arange(0,256), h)

t1 = time()
print("Time: ", t1 - to)

# codificar a mensagem
seq_bit0 = encode_huff(x, tabela_codigo)

# codificar tabela
seq_bit1 = encode_huff_table(tabela_codigo)

# concatenar sequencia de bits fa tabela e da mensagem
...

# escrver ficheiro
write2file(..., filename)
t2 = time()
print("Time: ", t2 - t1)

# ler ficheiro e descodificar a tabela
tabela_codigo, seq_bit1 = read_file(filename)

# descodificar mensagem
yi = decode_huff(seq_bit1, tabela_codigo)
t3 = time()
print("Time: ", t3 - t2)

size_ini = path.getsize("filename_original_image")
size_end = path.getsize("filename_compressed")
print("Taxa: ", 1. * size_ini/size_end)